In [2]:
import torch
from torch_geometric_temporal.dataset import METRLADatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

In [3]:
# Loading dataset

loader = METRLADatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio = 0.8)

In [4]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [73]:
# class for simple LSTM

class TrafficLSTM(nn.Module):
    def __init__(self, input_size: int, hidden_size:  int, num_layers: int, sequence_len: int):
        super().__init__()
        self.sequence_len = sequence_len
        self.input_size = input_size
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first = True)
        # self.linear1 = nn.Linear(hidden_size*sequence_len, int(sequence_len*input_size/2))
        # self.relu1 = nn.ReLU()
        # self.linear2 = nn.Linear(int(sequence_len*input_size/2), sequence_len*input_size)
        self.linear = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        #batch_size = x.shape[0]
        x, _ = self.lstm(x)
        #x = x.reshape(batch_size, -1)
        # #print("LSTM output: ", x.shape)
        # x = self.linear1(x)
        # #print("Linear output: ", x.shape)
        # x = self.relu1(x)
        # x = self.linear2(x)
        # x = x.reshape(batch_size, self.sequence_len, self.input_size)
        #x = x[:, -1, :]
        x = self.linear(x)
        return x

In [74]:
# we want to take in features and make tensors of each point
# here, we can decide how far out we want to predict
# train will still be an hour of historical data
# test initially contains next 12 timesteps
# choose just one 
def create_dataset(train, test, timestep_to_predict=1):
    train_data_x = np.array(train.features)
    # just get speed
    train_data_x = train_data_x[:,:,0,:]
    N, S, T = train_data_x.shape
    #print(N,S,T)
    #train_data_x = train_data_x.reshape((N, S*T))
    train_data_y = np.array(train.targets)
    #train_data_y = train_data_y[:,:,timestep_to_predict].reshape(N, S, 1)
    #train_data_y = train_data_y.reshape((N, S*T))
    x_train = torch.tensor(train_data_x).swapaxes(1,2)
    y_train = torch.tensor(train_data_y).swapaxes(1,2)
    max_speed = torch.max(x_train)
    min_speed = torch.min(x_train)
    x_train = (x_train - min_speed)/(max_speed - min_speed)
    y_train = (y_train - min_speed)/(max_speed - min_speed)
    #x_train = torch.tensor(train_data_x)
    #y_train = torch.tensor(train_data_y)
    print(x_train.shape)
    print(y_train.shape)
    return data.TensorDataset(x_train, y_train)

In [75]:
dataset = create_dataset(train_dataset, test_dataset)
loader = data.DataLoader(dataset, batch_size = 8, drop_last=True)
for idx, (data1, label) in enumerate(loader):
    if idx > 0:
        break
    print('label.shape: {}'.format(label.shape))
    print('label: {}'.format(label))
    print('data.shape: {}'.format(data1.shape))
# temp = next(iter(loader))
# print(len(temp))
# print(len(temp[0]))
# print(len(temp[0][0]))

torch.Size([27399, 12, 207])
torch.Size([27399, 12, 207])
label.shape: torch.Size([8, 12, 207])
label: tensor([[[0.8732, 0.9571, 0.8357,  ..., 0.7589, 1.0000, 0.9071],
         [0.8365, 0.8952, 0.9397,  ..., 0.7508, 0.9460, 0.8905],
         [0.9089, 0.9571, 0.7857,  ..., 0.7750, 0.9607, 0.8339],
         ...,
         [0.8786, 0.8857, 0.8875,  ..., 0.8625, 1.0000, 0.8321],
         [0.9032, 0.9159, 0.9190,  ..., 0.8889, 0.9619, 0.9000],
         [0.9286, 0.9000, 0.9857,  ..., 0.8857, 0.9161, 0.8679]],

        [[0.8365, 0.8952, 0.9397,  ..., 0.7508, 0.9460, 0.8905],
         [0.9089, 0.9571, 0.7857,  ..., 0.7750, 0.9607, 0.8339],
         [0.9540, 0.9365, 0.9730,  ..., 0.8730, 0.9746, 0.8778],
         ...,
         [0.9032, 0.9159, 0.9190,  ..., 0.8889, 0.9619, 0.9000],
         [0.9286, 0.9000, 0.9857,  ..., 0.8857, 0.9161, 0.8679],
         [0.7651, 0.9397, 0.9190,  ..., 0.9286, 0.8984, 0.8873]],

        [[0.9089, 0.9571, 0.7857,  ..., 0.7750, 0.9607, 0.8339],
         [0.9540, 0.

In [78]:
import time

def train(model, dataloader, loss_func, device, optimizer):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (data1, label) in enumerate(dataloader):
        #label = label[:,-1,:]
        label = label.to(device)
        data1 = data1.to(device)
        optimizer.zero_grad()
        
        out = None
        ###########################################################################
        # TODO: compute the logits of the input, get the loss, and do the         #
        # gradient backpropagation.
        ###########################################################################
        if(idx == 0):
            print("input shape: ", data1.shape)
            print("label shape: ", label.shape)
        out = model(data1)
        out = out.swapaxes(1,2)
        label = label.swapaxes(1,2)
        if(idx == 0):
            print("output shape, ", out.shape)
            print("out: ", out[0][0])
            print("label: ", label[0][0])
        loss = loss_func(out, label)
        loss.backward()
        ###########################################################################
        #                             END OF YOUR CODE                            #
        ###########################################################################
        
        optimizer.step()

        
        train_rmse = torch.sqrt(loss)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| rmse {:8.3f}'.format(epoch, idx, len(dataloader),
                                              train_rmse))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(model, dataloader, loss_func, device):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, data1) in enumerate(dataloader):
            label = label.to(device)
            data1 = data1.to(device)
            
            ###########################################################################
            # TODO: compute the logits of the input, get the loss.                    #
            ###########################################################################
            logits = model(data1)
            loss = loss_func(logits, label)
            ###########################################################################
            #                             END OF YOUR CODE                            #
            ###########################################################################
            
            total_acc += (logits.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [79]:
from torch.utils.data.dataset import random_split
import torch.nn.functional as F
#from torchtext.data.functional import to_map_style_dataset

assert torch.cuda.is_available()
# device = 'cuda'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyper parameters
epochs = 10 # epoch
lr = 0.001 # learning rate
input_size = 207
hidden_size = 128

###########################################################################
# TODO: Deinfe the classifier and loss function.
###########################################################################
model = TrafficLSTM(input_size=input_size, hidden_size=hidden_size, num_layers=3, sequence_len=12)
loss_func = F.mse_loss
###########################################################################
#                             END OF YOUR CODE                            #
###########################################################################

# copy the model to the specified device (GPU)
model = model.to(device)
        
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, 1e-8)
total_accu = None


for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model, loader, loss_func, device, optimizer)
    #accu_val = evaluate(model, valid_dataloader, loss_func, device)
    # if total_accu is not None and total_accu > accu_val:
    #     scheduler.step()
    # else:
    #     total_accu = accu_val
    # print('-' * 59)
    # print('| end of epoch {:3d} | time: {:5.2f}s | '
    #       'valid accuracy {:8.3f} '.format(epoch,
    #                                        time.time() - epoch_start_time,
    #                                        accu_val))
    # print('-' * 59)

input shape:  torch.Size([8, 12, 207])
label shape:  torch.Size([8, 12, 207])
output shape,  torch.Size([8, 207, 12])
out:  tensor([0.0632, 0.0631, 0.0602, 0.0588, 0.0589, 0.0577, 0.0561, 0.0544, 0.0530,
        0.0519, 0.0511, 0.0503], device='cuda:0', grad_fn=<SelectBackward0>)
label:  tensor([0.8732, 0.8365, 0.9089, 0.9540, 0.7982, 0.9190, 0.9127, 0.9018, 0.8875,
        0.8786, 0.9032, 0.9286], device='cuda:0')
| epoch   1 |   500/ 3424 batches | rmse    0.219
| epoch   1 |  1000/ 3424 batches | rmse    0.209
| epoch   1 |  1500/ 3424 batches | rmse    0.198
| epoch   1 |  2000/ 3424 batches | rmse    0.352
| epoch   1 |  2500/ 3424 batches | rmse    0.182
| epoch   1 |  3000/ 3424 batches | rmse    0.263
input shape:  torch.Size([8, 12, 207])
label shape:  torch.Size([8, 12, 207])
output shape,  torch.Size([8, 207, 12])
out:  tensor([0.7810, 0.8227, 0.8320, 0.6307, 0.4337, 0.8279, 0.8335, 0.8351, 0.8359,
        0.8359, 0.8359, 0.8359], device='cuda:0', grad_fn=<SelectBackward0>)
